In [ ]:
import sys
sys.path.append("./")

import scanpy as sc
import pandas as pd
from os import listdir
import numpy as np
import re
from scipy import sparse

In [ ]:
import csv

df = pd.read_csv("all.cell.annotation.meta.txt",delimiter='\t')
df.to_csv('celltypes.csv')

In [ ]:
meta = pd.read_csv("celltypes.csv")
meta

In [ ]:
final_adata = None

In [ ]:
for fl in listdir("./datas"):
    adata = sc.read_10x_h5(f"./datas/{fl}")
    indices = adata.obs.index.values
    newIndices = np.array([re.sub('-\d+', '', x) for x in indices], dtype=object)
    
        
    filtered_meta = meta[meta['sample'] == fl[:-3]]
    rows = filtered_meta.ID.values
    metaID = np.array([re.sub('_\d+', '', x) for x in rows], dtype=object)
    i = np.nonzero(newIndices[:, None] == metaID)[0]
    
    if(adata.var.shape[0] > 33538):
        new_adata = sc.AnnData(X = adata.X[i,:-1])
    else:
        new_adata = sc.AnnData(X = adata.X[i])
    
    
    new_adata.obs['ID'] = filtered_meta['ID'].values   
    new_adata.obs['sample'] = filtered_meta['sample'].values      
    new_adata.obs['sample_new'] = filtered_meta['sample_new'].values      
    new_adata.obs['condition'] = filtered_meta['group'].values      
    new_adata.obs['celltype'] = filtered_meta['celltype'].values      
    
    if(final_adata):
        final_adata = final_adata.concatenate(new_adata, index_unique=None)
    else:
        final_adata = new_adata


In [ ]:
final_adata.var = adata.var
final_adata.var

In [ ]:
final_adata.obs = final_adata.obs.drop(['batch'], axis=1)

In [ ]:
final_adata.obs = final_adata.obs[['ID', 'sample', 'sample_new', 'condition', 'celltype']]

In [ ]:
final_adata

In [ ]:
final_adata.write('covid_data.h5ad')